# EX8

A **LCL Motores Ltda.**, uma fábrica de motores especiais, recebeu recentemente **R$ 900.000,00** em pedidos de seus três tipos de motores. Cada motor necessita de determinado número de horas de trabalho no setor de montagem e acabamento. A LCL pode terceirizar parte da sua produção. A tabela a seguir resume esses dados. A fábrica deseja determinar quantos motores ela deve produzir internamente e quantos devem ser produzidos de forma terceirizada para atender à demanda de pedidos.

## Dados do Problema

| Modelo              | 1        | 2        | 3        | **Total**   |
|---------------------|----------|----------|----------|-------------|
| **Demanda**         | 3000 unid. | 2500 unid. | 500 unid. | 6000 unid.  |
| **Montagem**        | 1 h/unid. | 2 h/unid. | 0,5 h/unid. | 6000 h      |
| **Acabamento**      | 2,5 h/unid. | 1 h/unid. | 4 h/unid. | 10000 h     |
| **Custo de Produção** | R$ 50,00 | R$ 90,00 | R$ 120,00 |             |
| **Custo Terceirizado** | R$ 65,00 | R$ 92,00 | R$ 140,00 |             |

## Objetivo
Determinar a quantidade de motores a serem produzidos internamente e terceirizados para atender a todos os pedidos, minimizando os custos de produção e terceirização.

---


###  Modelo Matemático

### Modelo Formal do Problema

#### Índices:
- $( i )$: Modelo motores, onde $( i \in \{ \text{1}, \text{2}, \text{3} \} )$.
- $( j )$: Setores de produção, onde $( j \in \{ \text{Montagem}, \text{Acabamento}\} )$.

#### Parâmetros:
- $( t_{ij} )$: Tempo médio de operação (em horas) para produzir um motor do modelo $( i )$ no setor $( j )$.
- $( T_j )$: Tempo disponível no operação $( j )$.
- $( C_i )$: Custo unitário do tipo $( i )$ produção própria.
- $( CT_i )$: Custo unitário do tipo $( i )$ produção tercerizada.
- $( D_i )$: Demanda do motor do tipo $( i )$.


#### Variáveis de Decisão:
- $( x_i )$: Quantidade de motor do tipo $( i )$ a serem fabricados por produção própria.
- $( y_i )$: Quantidade de motor do tipo $( i )$ a serem fabricados por produção terceira.

#### Função Objetivo:
Minimizar custo:

$$
Z = \sum_{i} C_i \cdot x_i + CT_i \cdot y_i
$$

#### Restrições:

1. **Tempo disponível :**

$$
\sum_{i} t_{ij} \cdot x_i \leq T_j , \quad \forall j
$$

2. **Atendimento a Demanda :**

$$
 x_i + y_i = D_i , \quad \forall i
$$

3. **Não negatividade:**

$$
x_i \geq 0, \quad \forall i
$$


In [18]:
import pyomo.environ as pyo

In [19]:
# Criação do modelo
model = pyo.ConcreteModel()

In [20]:

# Conjuntos
model.P = pyo.Set(initialize=[1, 2, 3])  
model.S = pyo.Set(initialize=['Montagem', 'Acabamento'])  

In [21]:
# ------------------------------
# Parâmetros
# ------------------------------
C = {1: 50, 2: 90, 3: 120}
model.C = pyo.Param(model.P,initialize=C)

CT = {1: 65, 2: 92, 3: 140}  # Custo unitário produção terceirizada
model.CT = pyo.Param(model.P,initialize=CT)

D = {1: 3000, 2: 2500, 3: 500}  # Demanda de cada motor
model.D = pyo.Param(model.P,initialize=D)

tempo_operacao = {(1, 'Montagem'): 1, (1, 'Acabamento'): 2.5,
     (2, 'Montagem'): 2, (2, 'Acabamento'): 1,
     (3, 'Montagem'): 0.5, (3, 'Acabamento'): 4}
model.tempo_operacao = pyo.Param(model.P,model.S, initialize=tempo_operacao)

tempo_disponiveis = {'Montagem': 6000, 'Acabamento': 10000}
model.tempo_disponiveis = pyo.Param(model.S, initialize=tempo_disponiveis)



In [22]:

# ------------------------------
# Variáveis de Decisão
# ------------------------------
model.x = pyo.Var(model.P, domain=pyo.NonNegativeReals) 
model.y = pyo.Var(model.P, domain=pyo.NonNegativeReals) 

In [23]:

# ------------------------------
# Função Objetivo
# ------------------------------
def fo(model):
    return sum(model.C[j] * model.x[j] + model.CT[j] * model.y[j] for j in model.P)
model.objetivo = pyo.Objective(rule=fo, sense=pyo.minimize)


In [24]:
# ------------------------------
# Restrições
# ------------------------------


# 1. Restrições de capacidade por operação
def restricao_capacidade(model, i):
    return sum(model.tempo_operacao[j,i] * model.x[j] for j in model.P) <= model.tempo_disponiveis[i]
model.capacidade = pyo.Constraint(model.S, rule=restricao_capacidade)

# 2. Demanda
def restricao_demanda(model, i):
    return model.x[i]+model.y[i] == model.D[i]
model.capacidade = pyo.Constraint(model.P, rule=restricao_demanda)


'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>). This
is usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [25]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex8.lp", io_options={"symbolic_solver_labels": True})

('ex8.lp', 2443093637344)

In [ ]:
# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.objetivo)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)

LP   has 3 rows; 6 cols; 6 nonzeros
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [5e+01, 1e+02]
  Bound  [0e+00, 0e+00]
  RHS    [5e+02, 3e+03]
Presolving model
0 rows, 0 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-3); columns 0(-6); elements 0(-6) - Reduced to empty
Solving the original LP from the solution after postsolve
Model status        : Optimal
Objective value     :  4.3500000000e+05
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00

Status do solver: ok
Condição de terminação: optimal
Valor da Função Objetivo: R$ 435.000,00


In [29]:
for i in model.P:    
    val = pyo.value(model.x[i])
    val2 = pyo.value(model.y[i])
    
    print(f" \n Quantidade do produto {i} produção própria : {val:.2f}")
    print(f"  Quantidade do produto {i} produção terceirizada : {val2:.2f}")
    print(f"  Demanda atendida do produto {i} : {val + val2:.2f}")


 
 Quantidade do produto 1 produção própria : 3000.00
  Quantidade do produto 1 produção terceirizada : 0.00
  Demanda atendida do produto 1 : 3000.00
 
 Quantidade do produto 2 produção própria : 2500.00
  Quantidade do produto 2 produção terceirizada : 0.00
  Demanda atendida do produto 2 : 2500.00
 
 Quantidade do produto 3 produção própria : 500.00
  Quantidade do produto 3 produção terceirizada : 0.00
  Demanda atendida do produto 3 : 500.00
